In [35]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
import random

In [36]:
df = pd.read_csv("covid_analytics_clinical_data.csv")
for col in df.columns:
    arr = df[col].to_numpy()

    if isinstance(arr[df[col].first_valid_index()],str):
        df[col] = df[col].str.lower()
        arr = df[col].to_numpy()
        arr = pd.unique(arr)
        classifier = np.array([i for i in range(len(arr))])
        df[col]=df[col].replace(arr,classifier)
#arr = df['Severity'].to_numpy()
#arr = pd.unique(arr)
#df["Severity"]=df["Severity"].replace(arr,[1,3,2,-1,1,2,3,2,3,4])
#arr = df['Survivors'].to_numpy()
#arr = pd.unique(arr)
#df["Survivors"]=df["Survivors"].replace(arr,[0,1,2,-1])
#arr = df['Country'].to_numpy()
#arr = pd.unique(arr)
#df["Country"]=df["Country"].replace(arr,[1, 2, 3, 4,5,6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, -1])
df = df.fillna(-1)
df

,ID,SUB_ID,Country,Province/State,Study Pop Size (N),Proportion of values entered in row (%),Paper Title,Journal,Author,Source,...,Hypoproteinaemia,Acidosis,Hospital admission (%),ICU admission,Discharged (%),"ICU length of stay, days","Hospital length of stay, days",Days to Viral Clearance (Median),Mortality,Projected Mortality (accounting for patients not currently discharged)
0,1.0,0.0,0,0,191.0,0,0,0,0,0,...,0,0,1.0,0.26,0,0,0,20.0,0,0
1,1.0,1.0,0,0,54.0,0,0,0,0,0,...,1,1,1.0,0.72,1,0,1,7.5,1,1
2,1.0,2.0,0,0,137.0,0,0,0,0,0,...,2,2,1.0,0.08,2,1,2,20.0,2,2
3,2.0,0.0,0,0,199.0,1,1,1,1,1,...,3,3,1.0,-1.00,0,2,3,28.0,3,3
4,2.0,1.0,0,0,99.0,1,1,1,1,1,...,3,3,1.0,-1.00,3,3,4,28.0,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,202.0,2.0,2,56,99.0,28,175,40,88,202,...,3,3,1.0,0.32,71,0,28,-1.0,30,42
535,202.0,3.0,2,56,117.0,31,175,40,88,202,...,3,3,1.0,0.54,53,13,13,-1.0,24,57
536,202.0,4.0,2,56,247.0,74,175,40,88,202,...,3,3,1.0,0.39,47,0,28,-1.0,3,4
537,202.0,5.0,2,56,50.0,19,175,40,88,202,...,3,3,1.0,0.42,22,13,28,-1.0,38,25
